   # PIK3R1: Effects of Mutation

<b>Standard imports for playing with and plotting data frames.</b>

In [3]:
import pandas as pd
import numpy as np
import scipy.stats
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt

alt.renderers.enable('notebook')

RendererRegistry.enable('notebook')

<b>Import CPTAC data</b>

In [4]:
import CPTAC

Loading Clinical Data...
Loading Proteomics Data...
Loading Transcriptomics Data...
Loading CNA Data...
Loading Phosphoproteomics Data...
Loading Somatic Data...

 ******PLEASE READ******


In [5]:
somatic_mutations = CPTAC.get_somatic()
proteomics = CPTAC.get_proteomics()
phos = CPTAC.get_phosphoproteomics()

### List of proteins that interact with PIK3R1 (according to Uniprot)

In [6]:
protList = ['ABI1', 'ABL2', 'AR', 'CBL', 'EGFR', 'ERBB2', 'ERBB3', 'ESR1', 'FASLG', 'FGFR1', 'RRAS', 'NRAS', 'PTEN', 'IRS1', 'IRS2', 'KIT', 'HRAS', 'IRS1', 'PIK3R1', 'PIK3CA', 'PIK3CD' 'RPS6KB1', 'AKT1', 'MRAS']

### Proteome abundance of proteins that interact with PIK3R1

In [7]:
gene = 'PIK3R1'
sigList = [];

pcutoff = 0.05/len(protList)
#pcutoff = 1

genedf = somatic_mutations[gene].to_frame()
for protein in protList:
    if protein in proteomics.columns:
        proteindf = proteomics[protein].to_frame()
        proteindfName = protein + " protein"
        proteindf.columns = [proteindfName]
        cross = genedf.add(proteindf, fill_value=0).dropna(axis=0)
        mutated = cross.loc[cross[gene] == 1.0]
        wt = cross.loc[cross[gene] == 0.0]
        ttest = scipy.stats.ttest_ind(mutated[proteindfName], wt[proteindfName])
        if ttest[1] <= pcutoff:
            sigList.append(protein)
            print("Test for " + protein + ": ")
            print(ttest)

Test for IRS2: 
Ttest_indResult(statistic=-4.387580404733772, pvalue=3.20389610243306e-05)
Test for PIK3CA: 
Ttest_indResult(statistic=-3.4392766499323155, pvalue=0.0008580917991622051)


### List of significantly affected proteins

In [8]:
print(sigList)

['IRS2', 'PIK3CA']


### Phosphoproteome abundance of interacting proteins

In [29]:
phosProtList = ['PIK3R', 'PIK3CA', 'IRS', 'RUFY', 'RASD', 'APPL', 'NRAS', 'HRAS', 'KRAS',
           'FAM83', 'AKT', 'RPS6KB1', 'PTEN']

In [32]:
gene = 'PIK3R1'
genedf = somatic_mutations[gene].to_frame()
sigPhosResults = [];

for protein in phosProtList:
    sites = phos.filter(regex=protein)
    genedf = genedf.add(sites, fill_value=0)

mutated = genedf.loc[genedf[gene] == 1.0]
wt = genedf.loc[genedf[gene] == 0.0]

pcutoff = 0.05 / len(genedf.columns)
print(pcutoff)
#pcutoff = 1

for loc in genedf.columns:
    if not loc == gene:
        mutsitedf = mutated[[gene, loc]].dropna()
        wtsitedf = wt[[gene, loc]].dropna()
        ttest = scipy.stats.ttest_ind(mutsitedf[loc], wtsitedf[loc])
        if(ttest[1] <= pcutoff):
            sigPhosResults.append(loc)
            print('Results for ' + loc + ': ')
            print(ttest)
 

0.0004950495049504951
Results for IRS2-S1100: 
Ttest_indResult(statistic=-4.019029388226416, pvalue=0.0001505481884237998)
Results for IRS2-S731: 
Ttest_indResult(statistic=-4.2075513087547245, pvalue=8.333919696760065e-05)


### List of significantly affected phosphorylation sites

In [28]:
print(sigPhosResults)

['IRS2-S1100', 'IRS2-S731']
